In [1]:
# todo pip install pillow
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:

base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(2,activation='softmax')(x) #final layer with softmax activation

c:\users\kurs\appdata\local\programs\python\python36\lib\site-packages\keras_applications\mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '
W0718 20:06:49.147570  8908 deprecation_wrapper.py:119] From c:\users\kurs\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0718 20:06:49.163559  8908 deprecation_wrapper.py:119] From c:\users\kurs\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0718 20:06:49.219521  8908 deprecation_wrapper.py:119] From c:\users\kurs\appdata\local\programs\python\python36\lib\site-packages\keras\bac

17227776/17225924 [==============================] - 3s 0us/step


In [5]:
model=Model(inputs=base_model.input,outputs=preds)

for layer in model.layers[:20]: # don't train the very first 20 layers!
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [4]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('d:/trainT/', # this is where you specify the path to the main data folder
                                                 target_size=(224,224), # anpassung der bilder passiert automatisch: original 150x150 auf 224x224
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

val_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

val_generator=val_datagen.flow_from_directory('d:/testT/', # this is where you specify the path to the main data folder
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 class_mode='categorical')

PermissionError: [WinError 21] Das Gerät ist nicht bereit: 'd:/trainT/'

In [ ]:
base_model.summary()

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_data=val_generator,
                   validation_steps=val_generator.n // train_generator.batch_size,
                   epochs=5)

In [ ]:

import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
#plt.imshow(x[0,:,:,:]+0.5,aspect=0.9)

x,y = val_generator.next()
a = model.predict(x)
pred = a[:,0] > 0.5
np.mean((y[:,0] > 0.5) == pred)
